In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add
from magma.bit_vector import BitVector
from magma.testing.coroutine import check


@si.coroutine
def Counter(width, init=0, incr=1):
    value = bits(init, width)
    O = value
    cout = False
    while True:
        yield O, cout
        O = value
        value, cout = add(value, bits(incr, width), cout=True)
        
Counter4 = si.compile(Counter(4), file_name="build/silica_counter.py")
with open("build/silica_counter.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40


In [2]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()
main = icestick.main()
counter = Counter4()
m.wire(counter.O, main.J1[:4])
wire(bits(0, 4), main.J1[4:])

m.compile("build/silica_counter", main)

NameError: name 'wire' is not defined

In [ ]:
%%bash
yosys -p 'synth_ice40 -top main -blif build/silica_counter.blif' build/silica_counter.v
arachne-pnr -d 1k -o build/silica_counter.txt -p build/silica_counter.pcf build/silica_counter.blif
icepack build/silica_counter.txt build/silica_counter.bin
icetime -tmd hx1k build/silica_counter.txt

In [ ]:
from mantle import Counter
icestick = IceStick()
icestick.Clock.on()
for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()
main = icestick.main()
counter = Counter(4)
m.wire(counter.O, main.J1[:4])
wire(bits(0, 4), main.J1[4:])

m.compile("build/mantle_counter", main)

In [ ]:
%%bash
yosys -p 'synth_ice40 -top main -blif build/mantle_counter.blif' build/mantle_counter.v
arachne-pnr -d 1k -o build/mantle_counter.txt -p build/mantle_counter.pcf build/mantle_counter.blif
icepack build/mantle_counter.txt build/mantle_counter.bin
icetime -tmd hx1k build/mantle_counter.txt